# Boilerplate

In [67]:
from typing import NamedTuple

import kfp
import kfp.components as comp
from kfp.components import InputPath, OutputPath, load_component_from_file, load_component_from_url
from kfp.components import func_to_container_op
import kfp.dsl as dsl
from kfp.compiler import compiler
from pathlib import Path

from kubernetes.client import V1Toleration
import kubernetes as k8s

# First model on jetson

First lines of fastbook. Modifications:

* No `import *`. In KFP we need everything in one function, so everything has to be named explicitly.
* Setup functions are called as part of this fucntion.


In [68]:
def train_first_model():
    import fastbook
    fastbook.setup_book()
    
    #from fastbook import *
    
    from fastai.vision.all import cnn_learner,ImageDataLoaders, untar_data
    from fastai.vision.all import URLs,get_image_files,Resize,resnet34,error_rate
    path = untar_data(URLs.PETS)/'images'

    def is_cat(x): return x[0].isupper()
    dls = ImageDataLoaders.from_name_func(
        path, get_image_files(path), valid_pct=0.2, seed=42,
        label_func=is_cat, item_tfms=Resize(224))

    learn = cnn_learner(dls, resnet34, metrics=error_rate)
    learn.fine_tune(1)

Specialities executed during this pipeline:

* Shared memory is a problem on the jetson. It was not possible to fix this with default values from host side, so a separate shm has been mounted using KFP.
* Toleration and node selector select the right node in the cluster.
* **NPT WORKING** - Container gets killed due to insufficient memory

In [69]:
@dsl.pipeline(
    name = "First fastai model",
    description = "Calculatin the daily stuff as a pipeline"
)
def vaccination_pipeline():
    #vop = dsl.VolumeOp(
    #    name="volume_creation",
    #    resource_name="mypvc",
    #    size="1Gi"
    #)
    volume = kfp.dsl.PipelineVolume(name='shm-vol', empty_dir={'medium': 'Memory'})
    #volume = dsl.PipelineVolume(volume=k8s.client.V1Volume(
    #    name=f"test-storage",
    #    empty_dir=k8s.client.V1EmptyDirVolumeSource()))
    
    step = comp.create_component_from_func(
        train_first_model,
        base_image="haumdaucherdatalab/l4t-fastbook:latest")()
    step.add_toleration(V1Toleration(effect='NoSchedule',key='nvidia.com/gpu'))
    step.add_node_selector_constraint('nvidia.com/gpu','true')
    step.add_pvolumes({'/dev/shm': volume})

kfp_client = kfp.Client()
kfp.compiler.Compiler().compile(vaccination_pipeline,package_path="fastbook_01.tar.gz")